# Application of LLM-Augmented Knowledge Graphs for Wirearchy Management

### Universitat Oberta de Catalunya
### Data Science Master's Degree - Data Analysis and Big Data.<br>Final project

- Author: Xavier Ventura de los Ojos
- Project Supervisor: Francesc Julbe López
- Coordinating Professor: Albert Solé Ribalta
- Date of submission: 06/2024

## POC 2: How to enrich a KG extracting content from semistructured documents

### Overview

Once the initial graph is modeled and the nodes and relationships created from strucutured files, we propose enriching it by extracting additional information from semistructured documents.
This task is known as Named Entity Recognition or NER.

In this POC 2 we showcase how Large Language Models (LLM) can be used to perform this NER task.<br>
The process consists on sending the extraction instructions (prompt) including the document content (text only) and the schema of interest.
Following the template instruction, the model should result the content in the form of a python dict.
The structured output from the LLM is enriched and then saved in a JSON file.<br>
This file is used for model performance analysis and for the actual enrichment of the graph.

It is NOT in the scope of the POC 2:
- Process images or sounds as input for the model.
- Implement a Langchain (generic) Agent that would categorize the document and proxy it to the corresponding extractor chain / schema. Instead we will focus on a specific use case.

### Use case: Agreements.

To illustrate the end to end process from entity identification to graph enrichment, we focus on a specific scenario which is relevant to the graph dataset.

The documents used for this POC 2 are the agreements published in PDF format in the open [Registre de Convenis of the Generalitat de Catalunya](https://registredeconvenis.gencat.cat) 

The information about the agreements is already part of the graph in the form of nodes labeled "Agreement".
But these nodes are isolated from the other entities.

The goal of this POC 2 is to connect the isolated "Agreement" nodes to the rest of the Graph.
More especificaly the connection will happen by:

* Creating necessary new "Person" nodes.
* Creating the SIGNED relationship between the "Person" nodes and the agreement.
* Adding the role, organization and document properties to the SIGNED relationship.
* Creating new Groups if the organization does not exist already.
* Creating the REPRESENT relationship between the Person and the Organization or Group.
* Creating the SIGNED relationship between the Organization or Group and the Agreement.


Luckily the list of URL is already available in the "Agreement" nodes of the Graph (see: POC 1).

Sample document URL:

https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0126


> For the sake of concisseness and cost and time savings, we don't attempt to load all the available documents but only those related to the code 2022/9/0304. If needeed, it is straightforward to extend the scope by just calling the *query_agreement* function with other agreement codes or just change the cypher query as per requirements.




## Components to parse documents and enrich the graph

|Component| Type | Description|
|---|---|--|
|<br><strong>Components for document parsing / NER task</strong><br>|
|parse|function|Parse a document or URL content and returns a list of documents.|
|Document schema (pydantic):||
| - Signee|class|Describe what is a document signee.|
| - Agreement|class|Describe what information to extract from an agreement.|
|PROMPT|ChatPromptTemplate|Template with extraction instructions.|
|LLMS|dict|Langchain Chats in scope of the POC.|
|extract|function|Returns the structured content from a text document given a schema and saves it in JSON format.|
|extract_documents|procedure|Extracts the structured content from a list of documents and stores it in JSON files.|
|<br>**Components for graph enrichment**<br>|
|update_graph_agreement|procedure|Updates the Graph with the information extracted from one agreement.|
|process_json_files|procedure|Load the json files located in the specified folder into the Graph.|
    



### Initial setup 

The following API keys and Neo4j pwd are expected to be available as environment variables:

- OPENAI_API_KEY: OpenAI API KEY.
- ANTHROPIC_API_KEY: Anthropic API KEY.
- LANGCHAIN_API_KEY: (Recommended).
- NEO4J_PWD: Password of the Neo4j user.


Execute the following %pip commands to install required packages if needed.

> There is a challenge with pulling documents from "portaldogc.gencat.cat" because of old Cipher: AES256-SHA.<br>
Use the following cell to downgrade the urllib3 to version 1.26.15 if you want to pull documents from "portaldogc.gencat.cat".
>
> More info: https://github.com/urllib3/urllib3/issues/3100
> 
> % openssl s_client -connect portaldogc.gencat.cat:443 


In [1]:
# Import required modules

import requests
import os, glob

import magic
from langchain.document_loaders.parsers import BS4HTMLParser, PDFMinerParser
from langchain.document_loaders.parsers.generic import MimeTypeBasedParser
from langchain.document_loaders.parsers.txt import TextParser
from langchain_community.document_loaders import Blob
from langchain_community.callbacks.manager import get_openai_callback

from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

import json

from neo4j import GraphDatabase

import datetime, time

### Components for document parsing / NER task

In [2]:
# Define constants

NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j",  os.environ['NEO4J_PWD'])

# Folders
EXTRACTS_FOLDER = "poc2_extracts"

# Configure the parsers that you want to use per mime-type!
HANDLERS = {
    "application/pdf": PDFMinerParser(),
    "text/plain": TextParser(),
    "text/html": BS4HTMLParser(),
}


In [3]:
def parse(url=None,file_name=None, **kwargs):
    """Parses a URL or file and returns  

    Args:
        url: Url pointing to the web or resource to parse
        file_name: Name of the file to be parsed.

    Returns:
        A list of documents
    """
    
    
    # Open URL or document
    
    if url:
        response = requests.get(url)
        data = response.content
    if file_name:
        data = open(file_name,'rb').read()

    # Parse document based on the file type
    
    # Configure the parsers that you want to use per mime-type!
    """
    HANDLERS = {
        "application/pdf": PDFMinerParser(),
        "text/plain": TextParser(),
        "text/html": BS4HTMLParser(),
    }

    # Instantiate a mimetype based parser with the given parsers
    MIMETYPE_BASED_PARSER = MimeTypeBasedParser(
        handlers=HANDLERS,
        fallback_parser=None,
    )
    """
    mime = magic.Magic(mime=True)
    mime_type = mime.from_buffer(data)

    print("mime_type: ",mime_type, " size: ", len(data))
    # A blob represents binary data by either reference (path on file system)
    # or value (bytes in memory).
    
    blob = Blob.from_data(data=data, mime_type=mime_type,)

    parser = HANDLERS[mime_type]
    documents = parser.parse(blob=blob)
    
    return documents
        

### Define the document Schema

We use pydantyc to define the structure of the expected output.

In this POC we will parse the agreements to learn who signed them from the document contents.

For this we define the classes:

- Signee: To capture the information about the person who signs the agreement on behalf of an organization.
- Agreement: To capture the agreement title, date and the list of Signees.

In [4]:

class Signee(BaseModel):
    """Information about the person who signs the agreement on behalf of an organization."""

    # ^ Doc-string for the entity Signee.
    # This doc-string is sent to the LLM as the description of the schema Signee,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    organization: Optional[str] = Field(default=None, description="The name of the organization that the person represents if known")
    role: Optional[str] = Field(default=None, description="The role of the person within the organization if known")
    document: Optional[str] = Field(default=None, description="The document proving the person's role within the organization if known")
    

class Agreement(BaseModel):
    """Extracted data about an agreement and the people signing it."""

    # Creates a model so that we can extract multiple entities.
    # (description="List of Personnel Actions")
    title: Optional[str] = Field(default=None, description="Title of the agreement")
    # summary: Optional[str] = Field(default=None, description="The document summary")
    date: Optional[str] = Field(default=None, description="The date of the agreement in YYYY-MM-DD format")
    # CVE: Optional[str] = Field(default=None, description="Document code with the prefix 'CVE-DOGC' if known")
    people: List[Signee] = Field(description="List of the agreement participants if known")
    

### Extraction Prompt

We define a custom prompt to provide the extraction instructions and any additional context.
 1) You can add examples into the prompt template to improve extraction quality
 2) Introduce additional parameters to take context into account (e.g., include metadata
    about the document from which the text was extracted.)


In [5]:
PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "The text is about an agreement between diferent parties. "
            "Only extract information from the text. "
            "Do not translate the content, produce any summary using the language of the document. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

### Create the Chats to interact with the LLM Models (OpenAI and Anthropic)

In [6]:
# Create the chats to interact with the LLMs.
LLMS={}

# OpenAI models
LLMS["gpt-3.5-turbo"] = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
LLMS["gpt-4-turbo"]   = ChatOpenAI(temperature=0, model="gpt-4-turbo")
LLMS["gpt-4o"]   = ChatOpenAI(temperature=0, model="gpt-4o") # gpt-4o released 2024-05-13

# Anthropic models: https://docs.anthropic.com/en/docs/models-overview#model-comparison
LLMS["claude-3-haiku"] = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")
LLMS["claude-3-opus"]  = ChatAnthropic(temperature=0, model_name="claude-3-opus-20240229")

REQUESTS_PER_MINUTE = {"claude-3-haiku": 5, "claude-3-opus": 5}

## Extract document's content

In [7]:
%env LANGCHAIN_TRACING_V2 = true
%env LANGCHAIN_PROJECT = POC_2

env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_PROJECT=POC_2


Extract information from document and save it in JSON format

In [8]:
# Function to extract the schema entities from the document

def now():
    return datetime.datetime.now(datetime.UTC).strftime("%Y-%m-%dT%H:%M:%SZ")

def extract(document,model_name,schema=Agreement, verbose=False):
    """Returns the structured content from a text document given a schema and saves it in JSON format.

    The process consists on the following steps:
        - parse: extract the text from the document or url.
        - extract: perform the NER task invoking the LLM.
        - store: optionally store the extracted information in a JSON file.

    Args:
        document: dict with the following keys:
            "code": Agreement code.
            "url": Url of the agreement document.
            "json_file": Filename where the structured data with additional metadata will be stored
        model_name: langchain chat (LLM) perfoming the NER
        schema: definition of the structured data to extract (defaults to Agreement)
        verbose: flag to indicate whether to output details of the process

    Returns:
        A tuple with:
            - entities extracted from the document.
            - dict with the entities plus additional metadata.
        
    """
    
    doc = parse(**document)
    
    print(document)
    if verbose:
        print(len(doc), doc[0].metadata, len(doc[0].page_content))
        print(doc[0].page_content[:200]) 

    # Call LLM to extract entities from the document
    if verbose: print("\nCalling LLM:\n")

    runnable = PROMPT | LLMS[model_name].with_structured_output(schema=schema)
   
    text = doc[0].page_content

    # To save token usage and parse time we only send the first 5000 chars of the document 
    # where the signature information is usually found
    text = text[:5000]

    timestamp = now()
    start_time = time.time()
    with get_openai_callback() as cb:
        entities = runnable.invoke({"text": text},
                                   {"tags": ["POC2", model_name],
                                    "metadata": {"code":document["code"], "llm": model_name, 
                                                 "size": len(text), "output": document["json_file"]}})
    extract_time = time.time() - start_time
    
    # Add document medatata to the LLM results
    data = entities.dict()
    data["code"] = document["code"]
    data["url"] = document["url"]
    metadata = {"model_name": model_name, "start_time": timestamp, "total_time": extract_time}
    # Add token and cost information when available
    if cb.total_tokens > 0:
        metadata.update({"prompt_tokens": cb.prompt_tokens,
                        "completion_tokens": cb.completion_tokens,
                        "total_tokens": cb.total_tokens,
                        "total_cost": cb.total_cost})
    data["metadata"] = metadata
    
    # Save the results from the LLM into a JSON file if required
    if document["json_file"]:
        with open(document["json_file"],'w') as fp:
            json.dump(data, fp, ensure_ascii=False, indent=4)
            
    if verbose: print(data); print()
    
    return (entities, data)

In [9]:
# Extract the estructured content from the list of docs and stores it in a JSON file.

def extract_documents(doclist, folder, model_name, schema=Agreement, verbose=False):
    """Extracts the structured content from a list of documents and stores it in JSON files.

    The structured content is extracted using the indicated model_name. JSON files are saves in the indicated subfolder
    under the "poc2_extracts" folder.

    Args:
        doclist: List of dictionaries with the "document_url" and "code" to extract.
        folder: Subfolder under "poc2_extracts" where JSON files will be stored.
        model_name: Name of the LLMs to user for the NER.
        verbose: flag indicating whether to output the process progress. 

    Returns:
        nothing
    """
    
    json_path = os.path.join(EXTRACTS_FOLDER, folder)
    os.makedirs(json_path,exist_ok=True)

    sleep_between_questions = 60.0 / REQUESTS_PER_MINUTE.get(model_name) if REQUESTS_PER_MINUTE.get(model_name) else 0.0
      
    for idx,doc in enumerate(doclist):

        # A wait time might be needed to not breach the request per minute rate imposed by some LLMs.
        if idx > 0 and (sleep_between_questions - extract_time)>0:
            wait_s = sleep_between_questions - extract_time
            print("\n wait(s):", wait_s)
            time.sleep(wait_s)
        
        start_time = time.time()
        
        json_file = os.path.join(json_path, doc["code"].replace("/","_") + ".json")
        #print (a["code"],json_file,a["document_url"])
        
        extract({"url": doc["document_url"],"code":doc["code"], "json_file": json_file}, model_name, schema=schema, verbose=verbose)    

        extract_time = time.time() - start_time
        

In [10]:
def query_agreement(code):
    """Queries the Graph to get the list of agreement codes and documents that are part or related with the given agreement.

    One agreement can be related with other agreements.
    The function will return the list of all agreements with their code and the document url.
    The document URL is stored as the "document_url" property of the nodes with label "Agreement".

    Args:
        code: The agreement code.

    Returns:
        A list of neo4j._data.Record with the attributes "code" and "document_url"
    """
    # references: https://neo4j.com/docs/python-manual/current/query-simple/#_write_to_the_database
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH) 
    
    query=("MATCH (a:Agreement {code: $code})"
           " RETURN a.code as code,a.document as document_url"
           " UNION "
           " match (a:Agreement {code: $code})-[]-(b:Agreement) "
           " return b.code as code, b.document as document_url"
           " order by code "
              )    
    result = driver.execute_query(query,code = code)
    
    driver.close()
    return result.records


### Components to enrich graph
- update_graph_agreement
- process_json_files

In [11]:
def update_graph_agreement(driver, agreement):
    """Updates the Graph with the information extracted from the agreement.
    
    Using Cypher, the following transaction is performed:
    
        - Create any necessary new "Person" nodes.
        - Create the SIGNED relationship between the "Person" nodes and the agreement.
        - Add the role, organization and document properties to the SIGNED relationship.
        - Create new Groups if the organization does not exist already.
        - Create the REPRESENT relationship between the Person and the Organization or Group.
        - Create the SIGNED relationship between the Organization or Group and the Agreement.

    Args:
        driver: Connection with Neo4j database.
        agreement: Dict with the Agreement information to ingest. 

    """
    
    # Complete any missing attribute
    if "people" not in agreement: agreement["people"] = []

    # Only consider records where the person's name is known
    parameters= {
        "people": [p for p in agreement["people"] if p["name"]],
        "code": agreement.get("code"),
    }
    
    # Update the agreement connecting People, Orgs and Groups
    if parameters["code"]:
        print(parameters["code"])
        print()

        # The title and date properties of the Agreement node are NOT updated.
        query=("MATCH (a:Agreement {code: $code})"
#              " set d.title = $title, d.date = date($date)"
              " WITH a"
              " UNWIND $people AS person" 
              " WITH *, apoc.text.clean(replace(person.name,' i ',' ')) as pk"
              " MERGE (p:Person {pk: pk}) ON CREATE set p.name=replace(person.name,' i ',' ')"
              " MERGE (p)-[r:SIGNED]->(a) set r.role=person.role, r.organization=person.organization, r.document=person.document"
              )    

        result = driver.execute_query(query,parameters_ = parameters)
        print("Created {nodes_created} nodes in {time} ms.".format(
                nodes_created=result.summary.counters.nodes_created,
                time=result.summary.result_available_after))
        
        # Create missing Organizations (Groups) and relationships

        for person in parameters["people"]:
            if person.get("organization"):
                print(person)
                print()
                records, summary, keys = driver.execute_query("""
                MATCH (a:Agreement {code: $code})
                MATCH (p:Person {pk: apoc.text.clean(replace($person.name,' i ',' '))})
                OPTIONAL MATCH (o:Organization) WHERE apoc.text.clean(o.name)=apoc.text.clean($person.organization)
                FOREACH (org in CASE WHEN o is null then [] else [o] end |
                    MERGE (p)-[r:REPRESENT]->(org) ON CREATE set r.role=$person.role
                    MERGE (org)-[s:SIGNED]->(a)
                )
                FOREACH (i in CASE WHEN o is null then [1] else [] end |
                    MERGE (g:Group {pk: apoc.text.clean($person.organization)}) 
                        ON CREATE set g.name=$person.organization
                    MERGE (p)-[rg:REPRESENT]->(g) 
                        ON CREATE set rg.role=$person.role
                    MERGE (g)-[sg:SIGNED]->(a)
                )
                """, person=person, code=parameters["code"])

                for record in records: print(record)
                for key in keys: print(keys)
        return

In [12]:
def process_json_files(folder):
    """Load the json files located in the specified folder into the Graph.

    Load the json files by calling the update_graph_agreement procedure.
    
    Args:
        folder: Folder name and file selector (e.g. "run02_gpt4o/*.json").
    """
    
    # Connect to Neo4j
    driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH) 

    filelist = glob.glob(os.path.join(EXTRACTS_FOLDER, folder))
    for idx,filename in enumerate(filelist):
        print("file:", filename)
    
        with open(filename,'r') as fp:
            data = json.load(fp)
        
        update_graph_agreement(driver, data)

    driver.close()
        

## Process the documents for the agreement "2022/9/0304"

To test this POC we will use the agreement "[2022/9/0304](https://presidencia.gencat.cat/ca/ambits_d_actuacio/relacions-institucionals/registre-de-convenis-de-collaboracio-i-cooperacio/detall#2022/9/0304)".  
This agreement consists of 22 PDF documents published in the [Registre de convenis de col·laboració i cooperació](https://presidencia.gencat.cat/ca/ambits_d_actuacio/relacions-institucionals/registre-de-convenis-de-collaboracio-i-cooperacio/)

All the agreements and the URL pointing to the corresponding PDF document are available in the graph as a result of the POC1.

### Step 1: Retrieve the signees from the documents.

In this step the PDF file of each agreement is parsed and its content sent to the LLM in order to extract the signees information.
The extracted signee information together with some metadata is then stored in a JSON file for further analysis and the proper ingestion into the graph for enrichment.

In order to compare the performance of different LLMs, the process is executed 4 times on all documents in scope using the following models:

|Vendor| Model | Description |
|---|---|--|
|[OpenAI](https://openai.com)|[GPT-3.5 Turbo](https://platform.openai.com/docs/models/gpt-3-5-turbo)|The latest GPT-3.5 Turbo model with higher accuracy at responding in requested formats and a fix for a bug which caused a text encoding issue for non-English language function calls. Returns a maximum of 4,096 output tokens.|
|[OpenAI](https://openai.com)|[GPT-4o](https://platform.openai.com/docs/models/gpt-4o)|Our most advanced, multimodal flagship model that’s cheaper and faster than GPT-4 Turbo. Currently points to gpt-4o-2024-05-13.|
|[Anthropic](https://www.anthropic.com)|[Claude 3 Haiku](https://docs.anthropic.com/en/docs/models-overview)|Our most powerful model, delivering state-of-the-art performance on highly complex tasks and demonstrating fluency and human-like understanding|
|[Anthropic](https://www.anthropic.com)|[Claude 3 Opus](https://docs.anthropic.com/en/docs/models-overview)|Our fastest and most compact model, designed for near-instant responsiveness and seamless AI experiences that mimic human interactions|



Query the graph to get the list of agreements / documents related with "2022/9/0304":

In [13]:
doc_list = query_agreement("2022/9/0304")

In [14]:
for doc in doc_list: print(doc["code"],"\n", doc["document_url"])

2022/9/0304 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0126
2022/8/0004 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0127
2022/8/0005 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0128
2022/8/0006 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0129
2022/8/0007 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0130
2022/8/0008 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&numConveni=2022/P/0131
2022/8/0009 
 https://registredeconvenis.gencat.cat/drep_rccc/public/Convenis.do?accion=DownloadDocumentsConveni&numFila=0&n

#### Extract structured data from files:
Convert the following cells from *Raw* to *Code* to perform the actual extraction.

Else the JSON files with the results of each run are available in the *poc2_extracts* folder.


> **REMEMBER**: There are some costs associated to the LLMs usage (token consumption) for running the tests. (It should be less than one USD though). Actual token usage (and cost estimate) is available in the json files for OpenAI. Token usage per call is available in LangSmith.


### Step 2: Ingest the JSON files into the Graph

In this step we use the JSON files to enrich the Graph.<br>
We decide to use the files produced by GPT-4o only. The rationalle behind this decision is provided in the thesis report.



In [15]:
process_json_files("run02_gpt4o/*.json")

file: poc2_extracts/run02_gpt4o/2022_8_0041.json
2022/8/0041

Created 0 nodes in 3717 ms.
{'name': 'Juan Pérez', 'organization': 'Universidad Nacional Autónoma de México', 'role': 'Rector', 'document': 'Nombramiento Rectoral'}

{'name': 'María López', 'organization': 'Secretaría de Educación Pública', 'role': 'Secretaria', 'document': 'Nombramiento Oficial'}

file: poc2_extracts/run02_gpt4o/2022_8_0016.json
2022/8/0016

Created 0 nodes in 206 ms.
file: poc2_extracts/run02_gpt4o/2024_8_0051.json
2024/8/0051

Created 0 nodes in 5 ms.
{'name': 'José Antonio Aguilera Núñez', 'organization': 'DEUTSCHE BANK, SAE', 'role': 'apoderat mancomunat', 'document': None}

{'name': 'Jorge Mateo Saenz de Miera Alonso', 'organization': 'DEUTSCHE BANK, SAE', 'role': 'apoderat mancomunat', 'document': None}

file: poc2_extracts/run02_gpt4o/2024_8_0047.json
2024/8/0047

Created 0 nodes in 159 ms.
{'name': 'Raimon Royo Uño', 'organization': 'Arquia Bank, SA', 'role': 'Director General Adjunt', 'document': '

**POC 2 Notebook ends here.**